In [1]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd, numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

In [34]:
# Read in the data

test = pd.read_csv("../otherdset/data_test_sarcasm.csv")
train = pd.read_csv("../otherdset/data_train_sarcasm.csv", low_memory=False)
print('Number of rows in train:', len(train))
print('Number of rows in test:', len(test))
print('Number of NaNs in comments column: ', sum(pd.isnull(train['comment']) == True))
print('Number of NaNs in comments column: ', sum(pd.isnull(test['comment']) == True))

# Remove NaNs
test.dropna(subset=['comment'], inplace = True)
train.dropna(subset=['comment'], inplace = True)
print('Number of rows in train:', len(train))
print('Number of rows in test:', len(test))
print('Number of NaNs in comments column:', sum(pd.isnull(train['comment']) == True))
print('Number of NaNs in comments column:', sum(pd.isnull(test['comment']) == True))

Number of rows in train: 950000
Number of rows in test: 60826
Number of NaNs in comments column:  43
Number of NaNs in comments column:  11
Number of rows in train: 949957
Number of rows in test: 60815
Number of NaNs in comments column: 0
Number of NaNs in comments column: 0


In [36]:
train.head()

,label,comment
0,0,NC and NH.
1,0,You do know west teams play against west teams...
2,0,"They were underdogs earlier today, but since G..."
3,0,"This meme isn't funny none of the ""new york ni..."
4,0,I could use one of those tools.


Preprocessing Data: Using tf-idf

In [37]:
# Setting up inputs
train_X = train.comment
train_y = train.iloc[:, 0:1]
test_X = test.comment
test_y = np.array(test.label)

In [38]:
# Since we want a vector representation of all words, we need to take both testing and training and tfidf them
combined_text = train_X.append(test_X, ignore_index=True)

In [39]:
# Initialize the tf-idf matrix from sklearn
vectorizer = TfidfVectorizer(strip_accents='unicode',
                             analyzer='word',
                             lowercase=True, # Convert all uppercase to lowercase
                             stop_words='english', # Remove commonly found english words ('it', 'a', 'the') which do not typically contain much signal
                             max_df = 0.9) # Only consider words that appear in fewer than max_df percent of all documents          
tfidf_matrix = vectorizer.fit(combined_text) # fit tfidf to comments

In [40]:
# Transform test and train into a numerical representation of comments
train_features_X = tfidf_matrix.transform(train_X)
test_features_X = tfidf_matrix.transform(test_X)

Fitting RandomForestClassifer

In [41]:
# Initialize the model
clf = RandomForestClassifier()

In [ ]:
# Fit the net to the training data
clf.fit(train_features_X, train_y.values.ravel())

In [82]:
# Print accuracy score
labels = clf.predict(test_features_X)
print("accuracy score:", metrics.accuracy_score(test_y, labels))

accuracy: 0.5715
